# DC-Biking State Usage Prediction Based On Weather

### 15688 Team Project 

Bowen Yang, I-Huei Huang, Gilbert Gao

# Motivation
Traffic flow prediction can be difficult because of its randomness natural. However, if we can predict the bike requirements, it will be helpful to schedule bikes for a bike rental system. In our team project, we will leverage the dataset provided by [Capital Bikeshare](https://www.capitalbikeshare.com/) system located at Washington D.C. as well as the historical weather data queried from the [Weather Underground history API](https://www.wunderground.com/weather/api/d/docs?d=data/history) .

By combining these data together, we plan to build a machine learning model that take historical bike using data and weather information as input and make prediction on total number of bikes required in Washington D.C. based on features of weather and the location of the bike station.

# Report Contents
* [Prerequisite](#Prerequisite)
* [Phase 1: Clean out the history bike data](#Phase 1: Clean out the history bike data)
* [Phase 2: Collect weather information -- weather underground API](#Phase 2: Collect weather information -- weather underground API) 
* [Further work](#Further work)

# Prerequisite
Download [history data from 2011 to 2015](https://s3.amazonaws.com/capitalbikeshare-data/index.html) from [Capital BikeShare](https://www.capitalbikeshare.com/trip-history-data).

Since the dataset is located in s3 storage, after configure the AWS Command Line Interface, we can download the whole dataset in a single line:

`$ aws s3 cp s3://capitalbikeshare-data . --recursive`


# Phase 1: Clean out the history bike data

Then we can start to clean out the history bike data. 

First we need to parse the station information from the XML file downloaded in the website. 

In [1]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import glob
import re

In [2]:
soup = BeautifulSoup(open("stations.xml"), "lxml")
station_list = soup.find_all('station')

The `station_list` is where we save all the station tag informations. We can print one of them to see what's inside to make ourselves clear about how to parse it.

In [19]:
print station_list[0]

<station><id>1</id><name>Eads St &amp; 15th St S</name><terminalname>31000</terminalname><lastcommwithserver>1478040900207</lastcommwithserver><lat>38.858971</lat><long>-77.05323</long><installed>true</installed><locked>false</locked><installdate>0</installdate><removaldate></removaldate><temporary>false</temporary><public>true</public><nbbikes>6</nbbikes><nbemptydocks>8</nbemptydocks><latestupdatetime>1478040899170</latestupdatetime></station>


Then now we can generate a dataframe to get the needed information and save into a csv file.

In [5]:
result = []
for each_tag in station_list:
    station_dict = {}
    station_dict['id'] = each_tag.id.string
    station_dict['name'] = each_tag.find('name').string
    station_dict['terminalname'] = each_tag.terminalname.string
    station_dict['lat'] = float(each_tag.lat.string)
    station_dict['long'] = float(each_tag.find('long').string)
    result.append(station_dict)
station_df = pd.DataFrame.from_dict(result)

In [6]:
print station_df.head()
station_df.to_csv('station.csv', index=False)

  id        lat       long                                        name  \
0  1  38.858971 -77.053230                         Eads St & 15th St S   
1  2  38.857250 -77.053320                             18th & Eads St.   
2  3  38.856425 -77.049232                           20th & Crystal Dr   
3  4  38.860170 -77.049593                           15th & Crystal Dr   
4  5  38.857866 -77.059490  Aurora Hills Community Ctr/18th & Hayes St   

  terminalname  
0        31000  
1        31001  
2        31002  
3        31003  
4        31004  


Then we move to the raw bike data to parse the useful information. 
First, we need the help from `glob` to get all the data files:

In [8]:
all_historical = glob.glob("*-cabi-trip-history-data.csv")

For our analysis, we only care about when and where the bike riders start their travel. Therefore we filter out all the unuseful information for us to simplify the parsing data size.

In [9]:
historical_df_list = []
for each_file in all_historical:
    temp_df = pd.read_csv(each_file)
    temp_df.columns = [each.lower() for each in temp_df.columns]
    if 'start station number' in temp_df.columns:
        # This is a updated version of dataset
        temp_df = temp_df[['start date','start station','start station number']]
        historical_df_list.append(temp_df)
    else:
        # older ones 
        temp_df = temp_df[['start date','start station']]
        historical_df_list.append(temp_df)

The original data comes with multiple date format. To organize and for future merging usage, we need to parse the dates to have same output format.

In [10]:
# Parse the dates
for each_df in historical_df_list:
    each_df['start date'] =  pd.to_datetime(each_df['start date'], infer_datetime_format = True)

The other messy part is that some of the station name does not only contains the name, but also the changed station name (represented by '[]') and the station terminal name (represented by the '()'). So a parsing using regular expression is needed. 

The changed station is saved for future use.

In [16]:
station_change = []
for each_df in historical_df_list:
    # Filter out the changed names
    extract_start = each_df['start station'].str.extract('(?P<extracted>[^\[\]]+) *(?P<change>\[[^\[\]]+\])? *', expand=False)
    new_start = extract_start.extracted.apply(lambda x: str(x).rstrip())
    each_df['start station'] = new_start
    # Then get the station change
    if extract_start.shape[1] > 1:
        station_change.append(extract_start.dropna(how='any').drop_duplicates())

Now we started to merge the station information with the historical travel data.

In [17]:
station_df = pd.read_csv('station.csv')
print station_df.columns
useful_station_df = station_df[[u'lat', u'long', u'name', u'terminalname']]
print useful_station_df.dtypes
print historical_df_list[-1].dtypes

Index([u'id', u'lat', u'long', u'name', u'terminalname'], dtype='object')
lat             float64
long            float64
name             object
terminalname      int64
dtype: object
start date              datetime64[ns]
start station                   object
start station number             int64
dtype: object


The data sets are not consistant for all these years. Some of the 

In [20]:
combined = []
# Find the useful information from the station_df
for each_df in historical_df_list:
    if "start station number" in each_df.columns:
        # merge on the numbers
        merged = each_df.merge(useful_station_df, left_on='start station number', right_on='terminalname', how='inner')
        combined.append(merged[['start date','start station','terminalname', 'lat', 'long']])
    else:
        # have to merge on the names:
        merged = each_df.merge(useful_station_df, left_on='start station', right_on='name', how='inner')
        combined.append(merged[['start date','start station','terminalname', 'lat', 'long']])

After merging, we can `concat` the list of dataframes and generate a master dataset for future usage. 

In [21]:
total_df = pd.concat(combined)
print total_df.shape

(11815281, 5)


In [ ]:
total_df.to_csv("merged_data.csv", index = False, date_format ="%m/%d/%Y %H:%M")

So now we have a historical travel dataset ready for merge and future manipulation. 

We are curious about how the weather information have influence the usage of the bike trips. Since we now have the travel data, we can move to the weather data retrieving. 

# Phase 2: Collect weather information -- weather underground API 
We collect historical weather information from 2011 to 2015 by calling history weather API on [Weather Underground](https://www.wunderground.com/weather/api/d/docs?d=data/history).
After downloading [history data from 2011 to 2015](https://s3.amazonaws.com/capitalbikeshare-data/index.html) from [Capital BikeShare](https://www.capitalbikeshare.com/trip-history-data), and applied API form wunderground.com. We extract the start date of each trip and call the API:
    __http://api.wunderground.com/api/{APIkey}/history_YYYYMMDD/q/DC.json__
to get the weather information in that day. And then, we parse the json format to get the weather at an exact hour in that day.
The following is the python script we used in this process:

First we include the packages we need:

In [ ]:
import requests
import json
import csv
import time

Then, we extract the weather information from 2011 to 2015 by calling weather undergound history API.

In [ ]:
key = 'your_api_key'
years = ['2011', '2012', '2013', '2014', '2015']
seasons = ['Q1', 'Q2', 'Q3', 'Q4']

for y in years:
    for s in seasons:
        # create log file
        logfile = "weather/log_" + y + "_" + s + ".txt"
        log = open(logfile, "wb")
        
        # create read and write file
        rfile = 'data/' + y + '-' + s + '-cabi-trip-history-data.csv'
        wfile = 'weather/' + rfile.split('/')[1].split('.')[0] +  '-weather.csv'
        csvrfile = open(rfile, 'rb')
        csvwfile = open(wfile, 'wb')
        
        # read csv file
        reader = csv.DictReader(csvrfile)
        rfieldnames = reader.fieldnames
        
        # write csv file
        wfieldnames = ['Start date', 'Weather type', 'Temperature', 'Humidity', 'Wind speed']
        writer = csv.DictWriter(csvwfile, fieldnames=wfieldnames)
        writer.writeheader()
        
        # store the time we have already visited to increase performance
        visit = set()
        for row in reader:
            d = {}
            # check the format of the date is valid: MM/DD/YYYY hh:mm
            if '/' not in row['Start date']:
                date_and_time = row['Start date'].split()
                mdate = date_and_time[0]
                num = mdate.split('-')
                mdate = num[1] + '/' + num[2] + '/' + num[0]
                mtime = date_and_time[1]
                row['Start date'] = mdate + ' ' + mtime
                
            # extract the date we need to call weather API    
            t = row['Start date']
            t = t.split()

            try:
                date = t[0].split('/') 
                year = int(date[2])
                mon = int(date[0])
                mday = int(date[1])
                date = str(year) + '%02d' % mon + '%02d' % mday
                t = t[1].split(':')
                hour = int(t[0])
                key = date + ":" + str(hour)
                if key in visit:
                    continue
                visit.add(key)
            except:
                print row['Start date']
                continue
            
            # begin to call weather API
            get = False
            while not get:
                try:
                    url = 'http://api.wunderground.com/api/' + key + '/history_' + date + '/q/DC.json'
                    r = requests.get(url)
                    parsed_json = json.loads(r.text)
                    observations = parsed_json["history"]["observations"]
                    get = True
                except:
                    time.sleep(100)
        
            # find the weather information at the exact hour we need
            for info in observations:

                if int(info["date"]["year"]) == year and int(info["date"]["mon"]) == mon and int(
                        info["date"]["mday"]) == mday and int(info["date"]["hour"]) == hour:
                    try:
                        d['Weather type'] = info["icon"]
                        d['Temperature'] = info["tempm"]
                        d['Humidity'] = float(info["hum"]) / 100
                        d['Wind speed'] = info["wspdm"]
                    except:
                        print year, mon, mday, hour
                        print info["icon"]
                        print info["tempm"]
                        print info["hum"]
                        print info["wspdm"]
                    break
                
            d['Start date'] = row['Start date'].split(':')[0]
            
            # If the data is missing, we discard the record, else write it to output file
            if len(d) != 5:
                print date + ":" + str(hour)
                log.write(date + ":" + str(hour)+ "\n")
            else:
                writer.writerow(d)
                
        print y + "-" + s + " finished"
        log.write(y + "-" + s + " finished\n")

The weather information collected is with the following format:

|  Start date     |  Weather Type	|   Temperature   |    Humidity     |   Wind Speed    |
|:---------------:|:---------------:|:---------------:|:---------------:|:---------------:|
| 01/01/2014 0    |	    clear	    |      1.1	      |       0.46      |   0             |
| 01/01/2014 1	  |    partlycloudy |      -1.1	      |       0.61      |   13            |
| 01/01/2014 2	  |    partlycloudy |      -0.6	      |       0.57      |   5.6           |